# Query Pipeline for Advanced Text-to-SQL

In this guide we show you how to setup a text-to-SQL pipeline over your data with our [query pipeline](https://docs.llamaindex.ai/en/stable/module_guides/querying/pipeline/root.html) syntax.

This gives you flexibility to enhance text-to-SQL with additional techniques. We show these in the below sections:
1. **Query-Time Table Retrieval**: Dynamically retrieve relevant tables in the text-to-SQL prompt.
2. **Query-Time Sample Row retrieval**: Embed/Index each row, and dynamically retrieve example rows for each table in the text-to-SQL prompt.

Our out-of-the box pipelines include our `NLSQLTableQueryEngine` and `SQLTableRetrieverQueryEngine`. (if you want to check out our text-to-SQL guide using these modules, take a look [here](https://docs.llamaindex.ai/en/stable/examples/index_structs/struct_indices/SQLIndexDemo.html)). This guide implements an advanced version of those modules, giving you the utmost flexibility to apply this to your own setting.

## Load and Ingest Data


### Load Data
We use the [WikiTableQuestions dataset](https://ppasupat.github.io/WikiTableQuestions/) (Pasupat and Liang 2015) as our test dataset.

We go through all the csv's in one folder, store each in a sqlite database (we will then build an object index over each table schema).

In [11]:
import requests
import zipfile
import io
import os

In [ ]:
URL = "https://github.com/ppasupat/WikiTableQuestions/releases/download/v1.0.2/WikiTableQuestions-1.0.2-compact.zip"

OUTPUT_DIR = "../data"
os.makedirs(OUTPUT_DIR, exist_ok=True)

print("Downloading...")
response = requests.get(URL)
response.raise_for_status()

print("Extracting...")
with zipfile.ZipFile(io.BytesIO(response.content)) as z:
    z.extractall(OUTPUT_DIR)

print("Done.")

Downloading...
Extracting...
Done.


In [12]:
import pandas as pd
from pathlib import Path

In [13]:
DATA_DIR = Path("../data/WikiTableQuestions/csv/200-csv")
CSV_FILES = sorted([f for f in DATA_DIR.glob("*.csv")])
dfs = []

for csv_file in CSV_FILES:
    print(f"processing file: {csv_file}")
    try:
        df = pd.read_csv(csv_file)
        dfs.append(df)
    except Exception as e:
        print(f"Error parsing {csv_file}: {str(e)}")

processing file: ..\data\WikiTableQuestions\csv\200-csv\0.csv
processing file: ..\data\WikiTableQuestions\csv\200-csv\1.csv
processing file: ..\data\WikiTableQuestions\csv\200-csv\10.csv
processing file: ..\data\WikiTableQuestions\csv\200-csv\11.csv
processing file: ..\data\WikiTableQuestions\csv\200-csv\12.csv
processing file: ..\data\WikiTableQuestions\csv\200-csv\14.csv
processing file: ..\data\WikiTableQuestions\csv\200-csv\15.csv
Error parsing ..\data\WikiTableQuestions\csv\200-csv\15.csv: Error tokenizing data. C error: Expected 4 fields in line 16, saw 5

processing file: ..\data\WikiTableQuestions\csv\200-csv\17.csv
Error parsing ..\data\WikiTableQuestions\csv\200-csv\17.csv: Error tokenizing data. C error: Expected 6 fields in line 5, saw 7

processing file: ..\data\WikiTableQuestions\csv\200-csv\18.csv
processing file: ..\data\WikiTableQuestions\csv\200-csv\20.csv
processing file: ..\data\WikiTableQuestions\csv\200-csv\22.csv
processing file: ..\data\WikiTableQuestions\csv\20

### Extract Table Name and Summary from each Table

Here we use gpt-3.5 to extract a table name (with underscores) and summary from each table with our Pydantic program.

In [14]:
TABLEINFO_DIR = "../data/WikiTableQuestions_TableInfo"
os.makedirs(TABLEINFO_DIR, exist_ok=True)

In [15]:
from pydantic import BaseModel, Field
from llama_index.core import Settings
from llama_index.core.program import LLMTextCompletionProgram
from llama_index.llms.ollama import Ollama

In [16]:
class TableInfo(BaseModel):
    """Information regarding a structured table."""

    table_name: str = Field(
        ..., description="table name (must be underscores and NO spaces)"
    )
    table_summary: str = Field(
        ..., description="short, concise summary/caption of the table"
    )

PROMPT_STR = """\
Return only a JSON object, with no explanation, no prose, no markdown, and no trailing text.
You are to produce **only** a JSON object matching the following exact schema:

{
    "table_name": "<short_name_in_snake_case_without_spaces>",
    "table_summary": "<short concise caption of the table>"
}

Example:
{"table_name": "movie_info", "table_summary": "Summary of movie data"}

Rules:
- The table_name must be unique to the table, describe it clearly, and be in snake_case.
- Do NOT output a generic table name (e.g., "table", "my_table").
- Do NOT make the table name one of the following: {exclude_table_name_list}.
- Do NOT include any keys other than "table_name" and "table_summary".
- Do NOT include extra text before/after the JSON.
- Do NOT include any other keys or text before/after the JSON.
- Do NOT wrap in ```json.

Table:
{table_str}
"""

Settings.llm = Ollama(
    model="qwen3:0.6b", 
    request_timeout=240,
    format="json",
    # context_window=1000
)

program = LLMTextCompletionProgram.from_defaults(
    output_cls=TableInfo,
    prompt_template_str=PROMPT_STR,
    llm=Settings.llm,
    # verbose=True,
)

In [17]:
import json
import time
import re
import json as pyjson

In [19]:
def extract_first_json_block(text: str):
    match = re.search(r"\{.*\}", text, re.S)  # grab first {...} block
    if not match:
        raise ValueError("No JSON object found in output")
    return pyjson.loads(match.group())


MAX_RETRIES = 3


def _get_tableinfo_with_index(idx: int) -> str:
    results_gen = Path(TABLEINFO_DIR).glob(f"{idx}_*")
    results_list = list(results_gen)
    
    if len(results_list) == 0:
        return None
    elif len(results_list) == 1:
        path = results_list[0]
        json_str = path.read_text(encoding="utf-8")
        return TableInfo.model_validate_json(json_str)
    else:
        raise ValueError(f"More than one file matching index: {list(results_gen)}")


table_names = set()
table_infos = []

# for idx, df in enumerate(dfs):
#     table_info = _get_tableinfo_with_index(idx)
#     if table_info:
#         table_infos.append(table_info)
#         continue
#     else:
#         while True:
#             df_str = df.head(10).to_csv()
            
#             for attempt in range(MAX_RETRIES):
#                 try:
#                     raw_output = program(
#                         table_str=df_str,
#                         exclude_table_name_list=str(list(table_names)),
#                     )
                    
#                     if isinstance(raw_output, TableInfo):
#                         table_info = raw_output
#                     elif isinstance(raw_output, dict):
#                         table_info = TableInfo(**raw_output)
#                     elif isinstance(raw_output, str):
#                         parsed_dict = extract_first_json_block(raw_output)
#                         table_info = TableInfo(**parsed_dict)
#                     else:
#                         raise TypeError(f"Unexpected return type from program(): {type(raw_output)}")
                    
#                     break
#                 except Exception as e:
#                     print(f"Error with attempt {attempt+1}: {str(e)}")
#                     time.sleep(2)
#             else:
#                 raise RuntimeError("Failed after multiple retries")
            
#             table_name = table_info.table_name
#             print(f"Processed table: {table_name}")
#             if table_name not in table_names:
#                 table_names.add(table_name)
#                 break
#             else:
#                 # try again
#                 print(f"Table name {table_name} already exists, trying again.")
#                 pass

#         out_file = f"{TABLEINFO_DIR}/{idx}_{table_name}.json"
#         json.dump(table_info.model_dump(), open(out_file, "w"))
#     table_infos.append(table_info)

for idx, df in enumerate(dfs):
    table_info = _get_tableinfo_with_index(idx)
    if table_info:
        table_infos.append(table_info)
        continue

    df_str = df.head(10).to_csv()

    for attempt in range(MAX_RETRIES):
        try:
            raw_output = program(
                table_str=df_str,
                exclude_table_name_list=str(list(table_names)),
            )

            if isinstance(raw_output, TableInfo):
                table_info = raw_output
            elif isinstance(raw_output, dict):
                table_info = TableInfo(**raw_output)
            elif isinstance(raw_output, str):
                parsed_dict = extract_first_json_block(raw_output)
                table_info = TableInfo(**parsed_dict)
            else:
                raise TypeError(f"Unexpected return type from program(): {type(raw_output)}")

            table_name = table_info.table_name
            print(f"Processed table: {table_name}")

            if table_name in table_names:
                print(f"Table name '{table_name}' already exists, skipping this table.")
                table_info = None  # don’t append duplicate
                break  # skip

            # save table info
            table_names.add(table_name)
            out_file = f"{TABLEINFO_DIR}/{idx}_{table_name}.json"
            json.dump(table_info.model_dump(), open(out_file, "w"))
            break  # move to next table

        except Exception as e:
            print(f"Error with attempt {attempt+1}: {e}")
            time.sleep(2)

    if table_info:
        table_infos.append(table_info)

Processed table: movie_chart_positions
Error with attempt 1: 1 validation error for TableInfo
  Invalid JSON: trailing characters at line 1 column 70 [type=json_invalid, input_value='{"table_name": "film_dat...Summary of movie data"}', input_type=str]
    For further information visit https://errors.pydantic.dev/2.11/v/json_invalid
Processed table: movie_data
Processed table: death_accident_statistics
Processed table: award_data_1972
Error with attempt 1: 1 validation error for TableInfo
  Invalid JSON: trailing characters at line 6 column 1 [type=json_invalid, input_value='{\n  "table_name": "awar...ata from 1979 onwards"}', input_type=str]
    For further information visit https://errors.pydantic.dev/2.11/v/json_invalid
Processed table: award_data
Processed table: people_info
Processed table: broadcasting_info
Processed table: person_info
Processed table: chart_positions
Processed table: kodachrome_film_info
Processed table: bbc_radio_costs
Processed table: airport_locations
Processe

To retry for a single index (in needed)

In [ ]:
# idx = 20
# df = dfs[idx]

# table_info = _get_tableinfo_with_index(idx)
# if table_info:
#     table_infos.append(table_info)
# else:
#     df_str = df.head(20).to_csv()

#     for attempt in range(MAX_RETRIES):
#         try:
#             raw_output = program(
#                 table_str=df_str,
#                 exclude_table_name_list=str(list(table_names)),
#             )

#             if isinstance(raw_output, TableInfo):
#                 table_info = raw_output
#             elif isinstance(raw_output, dict):
#                 table_info = TableInfo(**raw_output)
#             elif isinstance(raw_output, str):
#                 parsed_dict = extract_first_json_block(raw_output)
#                 table_info = TableInfo(**parsed_dict)
#             else:
#                 raise TypeError(f"Unexpected return type from program(): {type(raw_output)}")

#             table_name = table_info.table_name
#             print(f"Processed table: {table_name}")

#             if table_name in table_names:
#                 print(f"Table name '{table_name}' already exists, skipping this table.")
#                 table_info = None
#                 break

#             table_names.add(table_name)
#             out_file = f"{TABLEINFO_DIR}/{idx}_{table_name}.json"
#             json.dump(table_info.model_dump(), open(out_file, "w"))
#             break

#         except Exception as e:
#             print(f"Error with attempt {attempt+1}: {e}")
#             time.sleep(2)

#     if table_info:
#         table_infos.append(table_info)

Error with attempt 1: 1 validation error for TableInfo
  Invalid JSON: trailing characters at line 1 column 97 [type=json_invalid, input_value='{"table_name": "award_in...n the specified table"}', input_type=str]
    For further information visit https://errors.pydantic.dev/2.11/v/json_invalid
Processed table: award_nominations


### Put Data in SQL Database

We use `sqlalchemy`, a popular SQL database toolkit, to load all the tables.

In [20]:
# put data into sqlite db
from sqlalchemy import (
    create_engine,
    MetaData,
    Table,
    Column,
    String,
    Integer,
)

In [25]:
# Function to create a sanitized column name
def sanitize_column_name(col_name):
    # Remove special characters and replace spaces with underscores
    return re.sub(r"\W+", "_", col_name)


# Function to create a table from a DataFrame using SQLAlchemy
def create_table_from_dataframe(
    df: pd.DataFrame, table_name: str, engine, metadata_obj
):
    # Sanitize column names
    sanitized_columns = {col: sanitize_column_name(col) for col in df.columns}
    df = df.rename(columns=sanitized_columns)

    # Dynamically create columns based on DataFrame columns and data types
    columns = [
        Column(col, String if dtype == "object" else Integer)
        for col, dtype in zip(df.columns, df.dtypes)
    ]

    # Create a table with the defined columns
    table = Table(table_name, metadata_obj, *columns)

    # Create the table in the database
    metadata_obj.create_all(engine)

    # Insert data from DataFrame into the table
    with engine.connect() as conn:
        for _, row in df.iterrows():
            insert_stmt = table.insert().values(**row.to_dict())
            conn.execute(insert_stmt)
        conn.commit()


engine = create_engine("sqlite:///:memory:")
metadata_obj = MetaData()
for idx, df in enumerate(dfs):
    tableinfo = _get_tableinfo_with_index(idx)
    if tableinfo is None:
        print(f"[ERROR] No TableInfo for index {idx}")
        continue  # skip this one or handle it differently
    print(f"Creating table: {tableinfo.table_name}")
    create_table_from_dataframe(df, tableinfo.table_name, engine, metadata_obj)

Creating table: movie_chart_positions
Creating table: movie_data
Creating table: death_accident_statistics
Creating table: award_data_1972
Creating table: award_data
Creating table: people_info
Creating table: broadcasting_info
Creating table: person_info
Creating table: chart_positions
Creating table: kodachrome_film_info
Creating table: bbc_radio_costs
Creating table: airport_locations
Creating table: party_voters
Creating table: club_performance
Creating table: horse_race_data
Creating table: grammy_awards
Creating table: boxing_matches
Creating table: sports_performance_data
Creating table: district_info
Creating table: party_data
Creating table: award_nominations
Creating table: government_ministers
Creating table: new_municipality_old_municipality_seat
Creating table: team_performance
Creating table: encoding_info
Creating table: temperature_data
Creating table: people_terms
Creating table: new_mexico_governorships
Creating table: weather_statistics
Creating table: drop_event_dat

In [27]:
# setup Arize Phoenix for logging/observability
import phoenix as px
from llama_index.core import set_global_handler

px.launch_app()
set_global_handler("arize_phoenix")

Existing running Phoenix instance detected! Shutting it down and starting a new instance...


🌍 To view the Phoenix app in your browser, visit http://localhost:6006/
📖 For more information on how to use Phoenix, check out https://arize.com/docs/phoenix


## Advanced Capability 1: Text-to-SQL with Query-Time Table Retrieval.

We now show you how to setup an e2e text-to-SQL with table retrieval.

### Define Modules

Here we define the core modules.
1. Object index + retriever to store table schemas
2. SQLDatabase object to connect to the above tables + SQLRetriever.
3. Text-to-SQL Prompt
4. Response synthesis Prompt
5. LLM

Object index, retriever, SQLDatabase

In [ ]:
from llama_index.core.objects import (
    SQLTableNodeMapping,
    ObjectIndex,
    SQLTableSchema,
)
from llama_index.core import SQLDatabase, VectorStoreIndex
from llama_index.embeddings.huggingface import HuggingFaceEmbedding

In [34]:
embed_model = HuggingFaceEmbedding(model_name="sentence-transformers/all-MiniLM-L6-v2")

sql_database = SQLDatabase(engine)
table_node_mapping = SQLTableNodeMapping(sql_database)

table_schema_objs = [
    SQLTableSchema(table_name=t.table_name, context_str=t.table_summary)
    for t in table_infos
]  # add a SQLTableSchema for each table

obj_index = ObjectIndex.from_objects(
    table_schema_objs,
    table_node_mapping,
    VectorStoreIndex,
    embed_model=embed_model,
)
obj_retriever = obj_index.as_retriever(similarity_top_k=3)

Xet Storage is enabled for this repo, but the 'hf_xet' package is not installed. Falling back to regular HTTP download. For better performance, install the package with: `pip install huggingface_hub[hf_xet]` or `pip install hf_xet`


SQLRetriever + Table Parser

In [36]:
from llama_index.core.retrievers import SQLRetriever
from typing import List

In [37]:
sql_retriever = SQLRetriever(sql_database)


def get_table_context_str(table_schema_objs: List[SQLTableSchema]):
    """Get table context string."""
    context_strs = []
    for table_schema_obj in table_schema_objs:
        table_info = sql_database.get_single_table_info(
            table_schema_obj.table_name
        )
        if table_schema_obj.context_str:
            table_opt_context = " The table description is: "
            table_opt_context += table_schema_obj.context_str
            table_info += table_opt_context

        context_strs.append(table_info)
    return "\n\n".join(context_strs)


table_parser_component = get_table_context_str(table_schema_objs)

Text-to-SQL Prompt + Output Parser

In [39]:
from llama_index.core.prompts.default_prompts import DEFAULT_TEXT_TO_SQL_PROMPT
from llama_index.core.prompts import PromptTemplate
# from llama_index.query_pipeline import FnComponent
from llama_index.core.tools import FunctionTool
from llama_index.core.llms import ChatResponse

In [40]:
def parse_response_to_sql(response: ChatResponse) -> str:
    """Parse response to SQL."""
    response = response.message.content
    sql_query_start = response.find("SQLQuery:")
    if sql_query_start != -1:
        response = response[sql_query_start:]
        # TODO: move to removeprefix after Python 3.9+
        if response.startswith("SQLQuery:"):
            response = response[len("SQLQuery:") :]
    sql_result_start = response.find("SQLResult:")
    if sql_result_start != -1:
        response = response[:sql_result_start]
    return response.strip().strip("```").strip()


sql_parser_tool = FunctionTool.from_defaults(fn=parse_response_to_sql)

text2sql_prompt = DEFAULT_TEXT_TO_SQL_PROMPT.partial_format(
    dialect=engine.dialect.name
)
print(text2sql_prompt.template)

Given an input question, first create a syntactically correct {dialect} query to run, then look at the results of the query and return the answer. You can order the results by a relevant column to return the most interesting examples in the database.

Never query for all the columns from a specific table, only ask for a few relevant columns given the question.

Pay attention to use only the column names that you can see in the schema description. Be careful to not query for columns that do not exist. Pay attention to which column is in which table. Also, qualify column names with the table name when needed. You are required to use the following format, each taking one line:

Question: Question here
SQLQuery: SQL Query to run
SQLResult: Result of the SQLQuery
Answer: Final answer here

Only use tables listed below.
{schema}

Question: {query_str}
SQLQuery: 


Response Synthesis Prompt

In [41]:
response_synthesis_prompt_str = (
    "Given an input question, synthesize a response from the query results.\n"
    "Query: {query_str}\n"
    "SQL: {sql_query}\n"
    "SQL Response: {context_str}\n"
    "Response: "
)
response_synthesis_prompt = PromptTemplate(
    response_synthesis_prompt_str,
)

In [ ]:
# llm = OpenAI(model="gpt-3.5-turbo")

Settings.llm = Ollama(
    model="qwen3:0.6b", 
    request_timeout=240,
    format="json",
    # context_window=1000
)

### Define Workflow

Now that the components are in place, let's define the query pipeline!

In [75]:
from llama_index.core.workflow import Workflow, step, StartEvent, Event, StopEvent


class Text2SQLWorkflow(Workflow):
    
    def __init__(self, **kwargs):
        super().__init__(**kwargs)
        
        self.connect(self.input_step, self.table_retriever_step)
        self.connect(self.table_retriever_step, self.table_output_parser_step)
        
        self.connect(self.input_step, self.text2sql_prompt_step)
        self.connect(self.table_output_parser_step, self.text2sql_prompt_step)

        self.connect(self.text2sql_prompt_step, self.text2sql_llm_step)
        self.connect(self.text2sql_llm_step, self.sql_output_parser_step)
        self.connect(self.sql_output_parser_step, self.sql_retriever_step)
        
        self.connect(self.sql_output_parser_step, self.response_synthesis_prompt_step)
        self.connect(self.sql_retriever_step, self.response_synthesis_prompt_step)
        self.connect(self.input_step, self.response_synthesis_prompt_step)
        
        self.connect(self.response_synthesis_prompt_step, self.response_synthesis_llm_step)
    
    @step()
    def input_step(self, event: StartEvent) -> Event:
        query = event.payload["query"]
        return Event(payload={"query_str": query})

    @step()
    def table_retriever_step(self, event: Event) -> Event:
        table_schema_objs = obj_retriever.retrieve(event.payload["query_str"])
        return Event(payload={"tables": table_schema_objs, **event.payload})

    @step()
    def table_output_parser_step(self, event: Event) -> Event:
        schema_str = get_table_context_str(event.payload["tables"])
        return Event(payload={"schema": schema_str, **event.payload})

    @step()
    def text2sql_prompt_step(self, event: Event) -> Event:
        prompt = text2sql_prompt.format(
            query_str=event.payload["query_str"],
            schema=event.payload.get("schema", "")
        )
        return Event(payload={"t2s_prompt": prompt, **event.payload})

    @step()
    def text2sql_llm_step(self, event: Event) -> Event:
        sql_query = Settings.llm(event.payload["t2s_prompt"])
        return Event(payload={"sql_query": sql_query, **event.payload})

    @step()
    def sql_output_parser_step(self, event: Event) -> Event:
        # If needed, parse or clean the SQL
        sql_query_clean = event.payload["sql_query"].strip()
        return Event(payload={"sql_query": sql_query_clean, **event.payload})

    @step()
    def sql_retriever_step(self, event: Event) -> Event:
        results = sql_retriever.retrieve(event.payload["sql_query"])
        return Event(payload={"context_str": results, **event.payload})

    @step()
    def response_synthesis_prompt_step(self, event: Event) -> Event:
        prompt = response_synthesis_prompt.format(
            query_str=event.payload["query_str"],
            context_str=event.payload.get("context_str", ""),
            sql_query=event.payload.get("sql_query", "")
        )
        return Event(payload={"rs_prompt": prompt, **event.payload})

    @step()
    def response_synthesis_llm_step(self, event: Event) -> StopEvent:
        answer = Settings.llm(event.payload["rs_prompt"])
        return StopEvent(payload={"answer": answer})

In [76]:
workflow = Text2SQLWorkflow()

AttributeError: 'Text2SQLWorkflow' object has no attribute 'connect'

### Visualize Workflow

A really nice property of the query pipeline syntax is you can easily visualize it in a graph via networkx.

In [62]:
import networkx as nx
from pyvis.network import Network

# Build a directed graph of steps
G = nx.DiGraph()

# Nodes
steps = [
    "input",
    "table_retriever",
    "table_output_parser",
    "text2sql_prompt",
    "text2sql_llm",
    "sql_output_parser",
    "sql_retriever",
    "response_synthesis_prompt",
    "response_synthesis_llm"
]
G.add_nodes_from(steps)

# Edges
edges = [
    ("input", "table_retriever"),
    ("table_retriever", "table_output_parser"),
    
    ("input", "text2sql_prompt"),
    ("table_output_parser", "text2sql_prompt"),

    ("text2sql_prompt", "text2sql_llm"),
    ("text2sql_llm", "sql_output_parser"),
    ("sql_output_parser", "sql_retriever"),
    
    ("sql_output_parser", "response_synthesis_prompt"),
    ("sql_retriever", "response_synthesis_prompt"),
    ("input", "response_synthesis_prompt"),
    
    ("response_synthesis_prompt", "response_synthesis_llm")
]
G.add_edges_from(edges)

# Visualize
net = Network(notebook=True, cdn_resources="in_line", directed=True)
net.from_nx(G)

html_content = net.generate_html()
with open("../outputs/trials/text2sql_dag.html", "w", encoding="utf-8") as f:
    f.write(html_content)

print("Saved text2sql_dag.html successfully.")

Saved text2sql_dag.html successfully.


### Run Some Queries!

Now we're ready to run some queries across this entire pipeline.

In [ ]:
"Show me total sales by region"

In [70]:
result = await workflow.run(
    payload={"query": "What was the year that The Notorious B.I.G was signed to Bad Boy?"}
)
print(result.payload["answer"])

WorkflowRuntimeError: Error in step 'response_synthesis_llm_step': 'rs_prompt'

In [ ]:
response = qp.run(
    query="What was the year that The Notorious B.I.G was signed to Bad Boy?"
)
print(str(response))

> Running module input with input: 
query: What was the year that The Notorious B.I.G was signed to Bad Boy?

> Running module table_retriever with input: 
input: What was the year that The Notorious B.I.G was signed to Bad Boy?

> Running module table_output_parser with input: 
table_schema_objs: [SQLTableSchema(table_name='Bad_Boy_Artists', context_str='List of artists signed to Bad Boy Records and their album releases'), SQLTableSchema(table_name='Renaissance_Discography', context_str='Summa...

> Running module text2sql_prompt with input: 
query_str: What was the year that The Notorious B.I.G was signed to Bad Boy?
schema: Table 'Bad_Boy_Artists' has columns: Act (VARCHAR), Year_signed (INTEGER), _Albums_released_under_Bad_Boy (VARCHAR), and foreign keys: . The table description is: List of artists signed to Bad Boy Rec...

> Running module text2sql_llm with input: 
messages: Given an input question, first create a syntactically correct sqlite query to run, then look at the results

In [ ]:
response = qp.run(query="Who won best director in the 1972 academy awards")
print(str(response))

In [ ]:
response = qp.run(query="What was the term of Pasquale Preziosa?")
print(str(response))

## 2. Advanced Capability 2: Text-to-SQL with Query-Time Row Retrieval (along with Table Retrieval)

One problem in the previous example is that if the user asks a query that asks for "The Notorious BIG" but the artist is stored as "The Notorious B.I.G", then the generated SELECT statement will likely not return any matches.

We can alleviate this problem by fetching a small number of example rows per table. A naive option would be to just take the first k rows. Instead, we embed, index, and retrieve k relevant rows given the user query to give the text-to-SQL LLM the most contextually relevant information for SQL generation.

We now extend our query pipeline.

In [ ]:
from llama_index.query_pipeline import QueryPipeline as QP
from llama_index.service_context import ServiceContext

qp = QP(verbose=True)
# NOTE: service context will be deprecated in v0.10 (though will still be backwards compatible)
service_context = ServiceContext.from_defaults(callback_manager=qp.callback_manager)

### Index Each Table

We embed/index the rows of each table, resulting in one index per table.

In [ ]:
from llama_index import VectorStoreIndex, load_index_from_storage
from sqlalchemy import text
from llama_index.schema import TextNode
from llama_index.storage import StorageContext
import os
from pathlib import Path
from typing import Dict


def index_all_tables(
    sql_database: SQLDatabase, table_index_dir: str = "table_index_dir"
) -> Dict[str, VectorStoreIndex]:
    """Index all tables."""
    if not Path(table_index_dir).exists():
        os.makedirs(table_index_dir)

    vector_index_dict = {}
    engine = sql_database.engine
    for table_name in sql_database.get_usable_table_names():
        print(f"Indexing rows in table: {table_name}")
        if not os.path.exists(f"{table_index_dir}/{table_name}"):
            # get all rows from table
            with engine.connect() as conn:
                cursor = conn.execute(text(f'SELECT * FROM "{table_name}"'))
                result = cursor.fetchall()
                row_tups = []
                for row in result:
                    row_tups.append(tuple(row))

            # index each row, put into vector store index
            nodes = [TextNode(text=str(t)) for t in row_tups]

            # put into vector store index (use OpenAIEmbeddings by default)
            index = VectorStoreIndex(nodes, service_context=service_context)

            # save index
            index.set_index_id("vector_index")
            index.storage_context.persist(f"{table_index_dir}/{table_name}")
        else:
            # rebuild storage context
            storage_context = StorageContext.from_defaults(
                persist_dir=f"{table_index_dir}/{table_name}"
            )
            # load index
            index = load_index_from_storage(
                storage_context, index_id="vector_index", service_context=service_context
            )
        vector_index_dict[table_name] = index

    return vector_index_dict


vector_index_dict = index_all_tables(sql_database)

Indexing rows in table: Academy_Awards_1972
Indexing rows in table: Actress_Awards
Indexing rows in table: Actress_Awards_Table
Indexing rows in table: Actress_Filmography
Indexing rows in table: Afrikaans_Language_Translations
Indexing rows in table: Airport_Information
Indexing rows in table: Average_Temperature_Precipitation
Indexing rows in table: Average_Temperature_and_Precipitation
Indexing rows in table: BBC_Radio_Costs
Indexing rows in table: Bad_Boy_Artists
Indexing rows in table: Boxing_Matches
Indexing rows in table: Club_Performance_Norway
Indexing rows in table: Disappeared_Persons
Indexing rows in table: Drop Events
Indexing rows in table: European_Football_Standings
Indexing rows in table: Football_Team_Records
Indexing rows in table: Gortynia_Municipalities
Indexing rows in table: Grammy_Awards
Indexing rows in table: Italian_Presidents
Indexing rows in table: Kentucky_Derby_Winners
Indexing rows in table: Kinase_Cancer_Relationships
Indexing rows in table: Kodachrome_

In [ ]:
test_retriever = vector_index_dict["Bad_Boy_Artists"].as_retriever(
    similarity_top_k=1
)
nodes = test_retriever.retrieve("P. Diddy")
print(nodes[0].get_content())

('Diddy', 1993, '6')


### Define Expanded Table Parser Component

We expand the capability of our `table_parser_component` to not only return the relevant table schemas, but also return relevant rows per table schema.

It now takes in both `table_schema_objs` (output of table retriever), but also the original `query_str` which will then be used for vector retrieval of relevant rows.

In [ ]:
from llama_index.retrievers import SQLRetriever
from typing import List
from llama_index.query_pipeline import FnComponent

sql_retriever = SQLRetriever(sql_database)


def get_table_context_and_rows_str(
    query_str: str, table_schema_objs: List[SQLTableSchema]
):
    """Get table context string."""
    context_strs = []
    for table_schema_obj in table_schema_objs:
        # first append table info + additional context
        table_info = sql_database.get_single_table_info(
            table_schema_obj.table_name
        )
        if table_schema_obj.context_str:
            table_opt_context = " The table description is: "
            table_opt_context += table_schema_obj.context_str
            table_info += table_opt_context

        # also lookup vector index to return relevant table rows
        vector_retriever = vector_index_dict[
            table_schema_obj.table_name
        ].as_retriever(similarity_top_k=2)
        relevant_nodes = vector_retriever.retrieve(query_str)
        if len(relevant_nodes) > 0:
            table_row_context = "\nHere are some relevant example rows (values in the same order as columns above)\n"
            for node in relevant_nodes:
                table_row_context += str(node.get_content()) + "\n"
            table_info += table_row_context

        context_strs.append(table_info)
    return "\n\n".join(context_strs)


table_parser_component = FnComponent(fn=get_table_context_and_rows_str)

### Define Expanded Query Pipeline

This looks similar to the query pipeline in section 1, but with an upgraded table_parser_component.

In [ ]:
from llama_index.query_pipeline import (
    QueryPipeline as QP,
    Link,
    InputComponent,
    CustomQueryComponent,
)

qp.add_modules({
    "input": InputComponent(),
    "table_retriever": obj_retriever,
    "table_output_parser": table_parser_component,
    "text2sql_prompt": text2sql_prompt,
    "text2sql_llm": llm,
    "sql_output_parser": sql_parser_component,
    "sql_retriever": sql_retriever,
    "response_synthesis_prompt": response_synthesis_prompt,
    "response_synthesis_llm": llm,
})

In [ ]:
qp.add_link("input", "table_retriever")
qp.add_link("input", "table_output_parser", dest_key="query_str")
qp.add_link(
    "table_retriever", "table_output_parser", dest_key="table_schema_objs"
)
qp.add_link("input", "text2sql_prompt", dest_key="query_str")
qp.add_link("table_output_parser", "text2sql_prompt", dest_key="schema")
qp.add_chain(
    ["text2sql_prompt", "text2sql_llm", "sql_output_parser", "sql_retriever"]
)
qp.add_link(
    "sql_output_parser", "response_synthesis_prompt", dest_key="sql_query"
)
qp.add_link(
    "sql_retriever", "response_synthesis_prompt", dest_key="context_str"
)
qp.add_link("input", "response_synthesis_prompt", dest_key="query_str")
qp.add_link("response_synthesis_prompt", "response_synthesis_llm")

In [ ]:
from pyvis.network import Network

net = Network(notebook=True, cdn_resources="in_line", directed=True)
net.from_nx(qp.dag)
net.show("text2sql_dag.html")

text2sql_dag.html


### Run Some Queries

We can now ask about relevant entries even if it doesn't exactly match the entry in the database.

In [ ]:
response = qp.run(
    query="What was the year that The Notorious BIG was signed to Bad Boy?"
)
print(str(response))

> Running module input with input: 
query: What was the year that The Notorious BIG was signed to Bad Boy?

> Running module table_retriever with input: 
input: What was the year that The Notorious BIG was signed to Bad Boy?

> Running module table_output_parser with input: 
query_str: What was the year that The Notorious BIG was signed to Bad Boy?
table_schema_objs: [SQLTableSchema(table_name='Bad_Boy_Artists', context_str='List of artists signed to Bad Boy Records and their album releases'), SQLTableSchema(table_name='Football_Team_Records', context_str='Summary...

> Running module text2sql_prompt with input: 
query_str: What was the year that The Notorious BIG was signed to Bad Boy?
schema: Table 'Bad_Boy_Artists' has columns: Act (VARCHAR), Year_signed (INTEGER), _Albums_released_under_Bad_Boy (VARCHAR), and foreign keys: . The table description is: List of artists signed to Bad Boy Rec...

> Running module text2sql_llm with input: 
messages: Given an input question, first create 